In [1]:
import os

colab = False
# #### COLAB ONLY CELL
if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    # drive.mount('/content/gdrive',force_remount=True)
    os.chdir("/content/gdrive/MyDrive/COMEXT")
    !pip install -q -r world-trade-thesis/requirements.txt
    !pwd
else:
    os.chdir("H:/My Drive/COMEXT")
    !pwd


import hashlib, random, math, sys, pickle, time, zipfile, io, json#, powerlaw #,eurostat
import pandas as pd
import numpy as np
import networkx as nx
from tqdm.notebook import tqdm, trange
from networkx.classes.function import degree
import matplotlib.pyplot as plt
from matplotlib import cm

sys.path.append('H:/My Drive/COMEXT/world-trade-thesis/src')
import graph_analysis as g

/h/My Drive/COMEXT


In [2]:
# This script is to run the SBM on all years, all products graphs
# !Rscript world-trade-thesis/src/complete_sbm.r

In [3]:
dfs_prod = {}
for f in tqdm(os.listdir("data-samples/sbm/v2/")):
    if "ini" not in f:
        df = pd.read_csv("data-samples/sbm/v2/"+f,na_filter=False)
        f_l = f.split("_")
        y = f_l[1][1:]
        p = f_l[2][1:3]
        # try:
        if p not in dfs_prod:
            dfs_prod[p] = df.set_index("country").rename({"block":y},axis=1)
        else:
            dfs_prod[p] = pd.concat([dfs_prod[p], df.set_index("country").rename({"block":y},axis=1)],axis=1)

In [4]:
for p in tqdm(dfs_prod):
    dfs_prod[p].to_csv(f"data-samples/sbm_allyears/v2/complete_allyears_p{p}.csv")

In [5]:
metrics = pd.read_parquet("data-samples/metrics/metrics_complete_V.parquet")

In [6]:
# m2 = metrics.set_index(["country","prod","year"])
# m2["sbm_block"] = 0
# sbm = dfs_prod["30"].reset_index().rename({"30":"prod"},axis=1).set_index(["country","prod"]).stack().rename("sbm_block")

In [7]:
metrics = metrics.sort_values(["country","prod","year"]).set_index(["country","prod","year"])

In [8]:
df_prods = None
for p in tqdm(dfs_prod):
    if df_prods is None:
        sbm = dfs_prod[p]
        sbm[p] = p
        df_prods = sbm.reset_index().rename({p:"prod"},axis=1).set_index(["country","prod"]).stack().rename("sbm_block")
    else:
        sbm = dfs_prod[p]
        sbm[p] = p
        df_prods = pd.concat([df_prods,sbm.reset_index().rename({p:"prod"},axis=1).set_index(["country","prod"]).stack().rename("sbm_block")])

In [9]:
sbms = df_prods.reset_index().rename({"level_2":"year"},axis=1).sort_values(["country","prod","year"]).reset_index(drop=True).set_index(["country","prod","year"]).values
metrics["sbm_block"] = sbms

In [10]:
metrics = metrics.reset_index()

In [11]:
metrics.to_parquet("data-samples/metrics/metrics_complete_V_sbm2.parquet")